# Properties

Properties are an advanced feature of Python that you won't see too often, but it's worth knowing what they look like in case you encounter them.

Previously, we discussed different ways of accessing a class's data attributes.  If all we need to do is get the value and set the value, it's more appropriate to access the attribute directly.  If we have to do extra processing or control access in some way, we can do that with getter and setter methods.

There may be situations in which you first write a program that accesses a data attribute directly, then change your mind.  Perhaps you realized that some basic processing is needed when setting a value, but you don't want to rewrite all the code in your larger program.  You would like to add the functionality of a setter method without changing the interface to your class.

Python allows us to do this with a feature called properties.  In the code below, suppose we created our Drone class with a non-hidden altitude attribute.

In [4]:
class Drone:
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
    
d1 = Drone(100)
print("The Drone's altitude is", d1.altitude)
d1.altitude = 300
print("The Drone's altitude is", d1.altitude)

The Drone's altitude is 100
The Drone's altitude is 300


We now realize that we want to make sure that altitude is never below zero.  Unfortunately, we already have a large program that accesses this attribute directly, so we don't want to switch to a setter method.  Instead, we can use a property.  We first make the attribute hidden and create getter and setter methods.  We then define a property using these methods.

In [9]:
class Drone:
    
    def __init__(self, altitude = 0):
        self.__altitude = altitude
        self.ascend_count = 0
    
    def fly(self):
        print("The drone is flying at", self.__altitude, "feet.")
    
    def ascend(self, change):
        self.__altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.__altitude
    
    def set_altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.__altitude = new_altitude
        
    altitude = property(get_altitude, set_altitude)
    
d1 = Drone(100)
print("The Drone's altitude is", d1.altitude)
d1.altitude = 300
print("The Drone's altitude is", d1.altitude)
d1.altitude = -10

The Drone's altitude is 100
The Drone's altitude is 300


Exception: Drone cannot have a negative altitude.

From outside the class, we access the property in exactly the same way as we would access a data attribute.  We put the name of the property after a period after the name of the Drone instance.  Behind the scenes, however, the getter and setter methods are called to actually do the work.

Should you use properties?  As with so many things, the answer boils down to readability.  The property syntax is simple, but it might mislead other programmers into thinking that nothing much is happening in the class.  If you put a lot of processing in your getter and setter methods, it's generally better to call those methods explicitly.  In our case, the additional processing we added is really minimal, so the property is a reasonable solution.

Below, we demonstrate a different syntax for defining properties, using decorators.  These are the lines begining with `@` that appear before method headers.  The `@property` decorator goes before the getter method.  The setter method gets a decorator with the name of the property followed by `.setter`.

In [10]:
class Drone:
    
    def __init__(self, altitude = 0):
        self.__altitude = altitude
        self.ascend_count = 0
    
    def fly(self):
        print("The drone is flying at", self.__altitude, "feet.")
    
    def ascend(self, change):
        self.__altitude += change
        self.ascend_count += 1
        
    @property
    def altitude(self):
        return self.__altitude
        
    @altitude.setter
    def altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.__altitude = new_altitude
        
    
d1 = Drone(100)
print("The Drone's altitude is", d1.altitude)
d1.altitude = 300
print("The Drone's altitude is", d1.altitude)
d1.altitude = -10

The Drone's altitude is 100
The Drone's altitude is 300


Exception: Drone cannot have a negative altitude.

## Class Methods and Static Methods

All the methods we've written so far have been instance methods.  They get bound to a particular instance and the self parameter always refers to that instance.  Most of the time, this is the only type of method you need, but it's worth noting that Python has two other types of methods.

The method types are:

- Instance methods
- Class methods
- Static methods

We create class methods and static methods using decorators.  These are those special instructions beginning with `@` that come before a function header.

When we create a class method, the first parameter of the method is bound to the class instead of the instance.  We can demonstrate that below.

In [14]:
class Drone:
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
    
    @classmethod
    def print_class(cls):
        print(cls)
        
    
d1 = Drone(100)
d1.print_class()
Drone.print_class()

<class '__main__.Drone'>
<class '__main__.Drone'>


Even when we call our method on a particular instance, the cls parameter is bound to the class, `Drone`.  As an aside, the name `cls` has no special meaning in Python.  Just like `self`, what's important is that it's the first parameter, but the convention is to name it `cls` so others know what you're doing.

You might wonder what a class method is good for.  We would typically use a class method to access class attributes.  For example, we might use one to get the total number of drones we created.

In [17]:
class Drone:
    
    __num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.__num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
    
    @classmethod
    def get_num_drones(cls):
        return cls.__num_drones
        
    
d1 = Drone(100)
print(d1.get_num_drones())
d2 = Drone(200)
print(d2.get_num_drones())

1
2


Notice that the class method only performs operations that pertain to the class as a whole, not individual instances.  We could have written the same code without declaring the method to be a class method, and not much would have to change.  Still, the `@classmethod` decorator alerts us to the fact that we're not affecting the given instance, so it can be good programming style.

The last method type are `staticmethods`. These methods don't have a reference to either an instance or a class. Because of this, they are very limited in what they can do.  We would typically use them for operations that are relevant to a class, but that don't actually affect any attributes.

For example, we might use a method to convert from meters to feet, since our altitude is measured in feet.  Such a method doesn't need a reference to the instance or the class, so we can declare it a static method.

In [18]:
class Drone:
    
    __num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.__num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
    
    @staticmethod
    def feet_from_meters(meters):
        return meters * 3.28084
        
    
d1 = Drone(100)
d1.altitude = Drone.feet_from_meters(200)
print(d1.altitude)

656.168


This method would work fine without the `@staticmethod` decorator, but the decorator alerts us to the fact that the instance and the class can't be changed by the method.  As you continue to learn about classes, you will start seeing places that class methods and static methods can be useful.